# Parse tables to database

https://nbviewer.jupyter.org/github/FinanceData/OpenDartReader/blob/master/docs/OpenDartReader_reference_manual.ipynb
https://nbviewer.jupyter.org/github/FinanceData/OpenDartReader/blob/master/docs/OpenDartReader_users_guide.ipynb

```
KOSPI
├── Company
│   ├── Code
│   └── Name
├── FS
│   ├── Code
│   ├── Rpid
│   └── Time
├── BS
│   ├── Code
│   ├── Rpid 
│   ├── Account
│   └── Value
└── IS
    ├── Code
    ├── Rpid 
    ├── Account
    └── Value

```

# Finance DataReader

In [1]:
import time
import yaml
from tqdm.notebook import tqdm
from collections import defaultdict

import numpy as np
import pandas as pd
import OpenDartReader
import FinanceDataReader as fdr

import psycopg
from psycopg import sql
from pathlib import Path
from tqdm import tqdm

In [2]:
# Load settings
src_path = Path('.').absolute().parent
with (src_path / 'setting_files' / 'settings.yml').open('r') as file:
    settings = yaml.full_load(file)

In [3]:
dart_settings = settings['opendart']
dart = OpenDartReader(dart_settings['apikey'])

stocks = fdr.StockListing('KOSPI')
stocks = stocks.loc[~stocks['Sector'].isnull(), :]
stocks_syms = stocks['Symbol'].values
stocks = stocks.reset_index(drop=True)
stocks.fillna('', inplace=True)
for c in stocks.columns:
    print(c, stocks[c].dtype)

Symbol object
Market object
Name object
Sector object
Industry object
ListingDate datetime64[ns]
SettleMonth object
Representative object
HomePage object
Region object


In [4]:
db_settings = settings['db']

In [6]:
with psycopg.connect(user=db_settings['user'], password=db_settings['password'], autocommit=True) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT EXISTS( SELECT 1 FROM pg_database WHERE datname='{db_settings['dbname']}');")
        exists = cur.fetchone()[0]
    if not exists:
        with conn.cursor() as cur:
            cur.execute(sql.SQL("CREATE DATABASE {};").format(sql.Identifier(db_settings['dbname'])))
    else:
        print(f"Existence of Database {db_settings['dbname']}: {exists}")

Existence of Database fsqa: True


In [19]:
# if the database exists: create tables
commands = (
    """
    CREATE TABLE company (
        c_id SERIAL PRIMARY KEY,
        code VARCHAR(10) UNIQUE NOT NULL,
        name VARCHAR(255) NOT NULL,
        sector VARCHAR(255) NOT NULL
    )
    """,
    """ 
    CREATE TABLE financial_statement (
        fs_id SERIAL PRIMARY KEY,
        code VARCHAR(10) NOT NULL,
        rpid VARCHAR(30) UNIQUE NOT NULL,
        bsns_year INT NOT NULL,
        FOREIGN KEY (code) REFERENCES company (code) ON DELETE CASCADE
    )
    """,
    """
    CREATE TABLE balance_sheet (
        bs_id SERIAL PRIMARY KEY,
        code VARCHAR(10) NOT NULL,
        rpid VARCHAR(30) NOT NULL,
        quarter VARCHAR(10) NOT NULL,
        account VARCHAR(255) NOT NULL,
        value BIGINT NOT NULL,
        FOREIGN KEY (code) REFERENCES company (code) ON DELETE CASCADE,
        FOREIGN KEY (rpid) REFERENCES financial_statement (rpid) ON DELETE CASCADE
    )
    """,
    """
    CREATE TABLE income_statement (
        is_id SERIAL PRIMARY KEY,
        code VARCHAR(10) NOT NULL,
        rpid VARCHAR(30) NOT NULL,
        quarter VARCHAR(10) NOT NULL,
        account VARCHAR(255) NOT NULL,
        value BIGINT NOT NULL,
        FOREIGN KEY (code) REFERENCES company (code) ON DELETE CASCADE,
        FOREIGN KEY (rpid) REFERENCES financial_statement (rpid) ON DELETE CASCADE
    )
    """
)

conn = None
table_names = ['company', 'financial_statement', 'balance_sheet', 'income_statement', 'cashflow_statement']
with psycopg.connect(
        dbname=db_settings['dbname'],
        host=db_settings['host'],
        user=db_settings['user'], 
        password=db_settings['password'],
        port=db_settings['port']
    ) as conn:
    with conn.cursor() as cur:
        for i, command in enumerate(commands):
            cur.execute(
                f"SELECT EXISTS( SELECT 1 FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_names[i]}');"
            )
            exists = cur.fetchone()[0]
            if not exists:     
                cur.execute(sql.SQL(command))
                print(f'Table created: {table_names[i]}')
            else:
                print(f'Table already exists: {table_names[i]}')
    conn.commit()

Table created: company
Table created: financial_statement
Table created: balance_sheet
Table created: income_statement


In [8]:
table_columns = [
    ('c_id', 'code', 'name', 'sector'),
    ('fs_id', 'code', 'rpid', 'bsns_year'),
    ('bs_id', 'code', 'rpid', 'quarter', 'account', 'value'),
    ('is_id', 'code', 'rpid', 'quarter', 'account', 'value'),
]
tables = dict(zip(table_names, table_columns))
tables

{'company': ('c_id', 'code', 'name', 'sector'),
 'financial_statement': ('fs_id', 'code', 'rpid', 'bsns_year'),
 'balance_sheet': ('bs_id', 'code', 'rpid', 'quarter', 'account', 'value'),
 'income_statement': ('is_id', 'code', 'rpid', 'quarter', 'account', 'value')}

## Company

At current, we'll do only for Samsung and it's peer companies(in the same Sector)

In [9]:
stocks_syms = stocks.loc[:, 'Symbol'].values
corp = '005930'
# same_sector_companies = ['삼성전자', 'SK하이닉스', 'DB하이텍']
# same_sector_stocks = stocks.loc[stocks['Name'].isin(same_sector_companies)]
stocks.loc[stocks['Symbol']==corp, ['Symbol', 'Name', 'Sector']]

,Symbol,Name,Sector
705,005930,SamsungElec,통신 및 방송 장비 제조업


In [20]:
with psycopg.connect(
        dbname=db_settings['dbname'],
        host=db_settings['host'],
        user=db_settings['user'], 
        password=db_settings['password'],
        port=db_settings['port']
    ) as conn:
    with conn.cursor() as cur:
        for i, line in stocks.loc[stocks['Symbol']==corp, ['Symbol', 'Name', 'Sector']].iterrows():
            cur.execute(
                sql.SQL("INSERT INTO {}({}, {}, {}, {}) VALUES (DEFAULT, %s, %s, %s) ;").format(
                    sql.Identifier('company'), *list(map(sql.Identifier, tables['company']))
                ),
                tuple(line.values)
            )

## Financial Statements 

In [15]:
def convert_to_q(report_code, bsns_year, bsns_year_dict):
    report_code_word_dict = {
        '11011': ('', '4Q'), 
        '11013': (' 1분기', '1Q'), 
        '11012': (' 반기', '2Q'), 
        '11014': (' 3분기', '3Q')
    }

    s = f'제 {bsns_year_dict[bsns_year]} 기'
    w, q = report_code_word_dict[report_code]
    if report_code == '11011':
        return {s+w:q}
    else:
        return {s+w+'말':q, s+w:q}

# stocks_syms = stocks.loc[stocks['Symbol']==corp, 'Symbol'].values
data_path = src_path / 'data'
df_account = pd.read_csv(data_path / 'AccountName.csv')
account_eng_dict = dict(zip(df_account['acc_name_kor'].values, df_account['acc'].values))

bsns_years = list(range(2015, 2022))
report_codes = ['11011', '11013', '11012', '11014']
cols = ['rcept_no', 'thstrm_nm', 'account_nm', 'thstrm_amount']
bsns_year_dict = dict(zip(bsns_years, range(47, 47+len(bsns_years))))
accounts_dict = {
    'balance_sheet' : [
        '유동자산', '현금및현금성자산', '매출채권', '선급비용', '재고자산', '비유동자산', '유형자산', '무형자산', '자산총계',
        '유동부채', '매입채무', '단기차입금', '선수금', '비유동부채', '사채', '장기차입금', '부채총계', '자본총계', '부채와자본총계'
    ],
    'income_statement': [
        '수익(매출액)', '매출원가', '매출총이익', '판매비와관리비', '영업이익', '금융수익', '금융비용',
        '법인세비용차감전순이익(손실)', '법인세비용', '계속영업이익(손실)', '당기순이익(손실)'
    ]
}
data = defaultdict(list)
for bsns_year in bsns_years:
    for report_code in report_codes:
        df = dart.finstate_all(corp, bsns_year, reprt_code=report_code, fs_div='CFS')
        time.sleep(0.5)
        if df is None:
            continue
        
        # rcept_no
        recpt_no = df['rcept_no'].values[0]
        q_dict = convert_to_q(report_code, bsns_year, bsns_year_dict)
        # bs
        sj = 'BS'
        df_bs = df.loc[(df['sj_div'] == sj), cols]    #  & df['account_nm'].isin(accounts_dict['balance_sheet'])     
        df_bs.loc[:, 'thstrm_nm'] = df_bs['thstrm_nm'].apply(q_dict.get)
        # is
        sj = 'CIS' if df['sj_div'].isin(['IS']).sum() == 0 else 'IS'
        df_is = df.loc[(df['sj_div'] == sj), cols] # & df['account_nm'].isin(accounts_dict['income_statement'])
        df_is.loc[:, 'thstrm_nm'] = df_is['thstrm_nm'].apply(q_dict.get)
        data['info'].append([corp, recpt_no, f'{bsns_year}'])
        data['bs'].append(df_bs)
        data['is'].append(df_is)

# save
data['info'] = pd.DataFrame(data['info'], columns=['code', 'rpid', 'time'])
data['BS'] = pd.concat(data['bs']).reset_index(drop=True)
data['IS'] = pd.concat(data['is']).reset_index(drop=True)

data['BS'].loc[:, 'account_nm'] = data['BS']['account_nm'].apply(account_eng_dict.get)
data['IS'].loc[:, 'account_nm'] = data['IS']['account_nm'].apply(account_eng_dict.get)

data['BS'] = data['BS'].loc[~data['BS'].loc[:, 'account_nm'].isnull(), :].reset_index(drop=True)
data['IS'] = data['IS'].loc[~data['IS'].loc[:, 'account_nm'].isnull(), :].reset_index(drop=True)

In [16]:
# check if there is null
print(data['BS'].isnull().sum())
print(data['IS'].isnull().sum())

rcept_no         0
thstrm_nm        0
account_nm       0
thstrm_amount    0
dtype: int64
rcept_no         0
thstrm_nm        0
account_nm       0
thstrm_amount    0
dtype: int64


In [21]:
with psycopg.connect(
        dbname=db_settings['dbname'],
        host=db_settings['host'],
        user=db_settings['user'], 
        password=db_settings['password'],
        port=db_settings['port']
    ) as conn:
    with conn.cursor() as cur:
        for i, line in data['info'].iterrows():
            cur.execute(
                sql.SQL("INSERT INTO {}({}, {}, {}, {}) VALUES (DEFAULT, %s, %s, %s) ;").format(
                    sql.Identifier('financial_statement'), *list(map(sql.Identifier, tables['financial_statement']))
                ),
                tuple(line.values)
            )

        for i, line in data['BS'].iterrows():
            cur.execute(
                sql.SQL("INSERT INTO {}({}, {}, {}, {}, {}, {}) VALUES (DEFAULT, %s, %s, %s, %s, %s) ;").format(
                    sql.Identifier('balance_sheet'), *list(map(sql.Identifier, tables['balance_sheet']))
                ),
                tuple([corp] + list(line.values))
            )

        for i, line in data['IS'].iterrows():
            cur.execute(
                sql.SQL("INSERT INTO {}({}, {}, {}, {}, {}, {}) VALUES (DEFAULT, %s, %s, %s, %s, %s) ;").format(
                    sql.Identifier('income_statement'), *list(map(sql.Identifier, tables['income_statement']))
                ),
                tuple([corp] + list(line.values))
            )

        # CREATE VIEW TABLE
        # VIEW TABLE: Income Statement
        cur.execute(
        """
        CREATE VIEW vt_is_005930
        AS 
        SELECT fs.bsns_year AS bsns_year, inc.quarter AS quarter, inc.account AS account, inc.value AS value
        FROM income_statement AS inc
        INNER JOIN company AS com
            ON com.code = inc.code
        INNER JOIN financial_statement AS fs
            ON fs.code = inc.code AND fs.rpid = inc.rpid
        WHERE
            com.name = 'SamsungElec' AND fs.bsns_year BETWEEN 2016 AND 2021;
        """)
        # VIEW TABLE: Balance Sheet
        cur.execute("""
        CREATE VIEW vt_bs_005930
        AS 
        SELECT fs.bsns_year AS bsns_year, bal.quarter AS quarter, bal.account AS account, bal.value AS value
        FROM balance_sheet AS bal
        INNER JOIN company AS com
            ON com.code = bal.code
        INNER JOIN financial_statement AS fs
            ON fs.code = bal.code AND fs.rpid = bal.rpid
        WHERE
            com.name = 'SamsungElec' AND fs.bsns_year BETWEEN 2016 AND 2021;
        """)

---

In [28]:
tables

{'company': ('c_id', 'code', 'name', 'sector'),
 'financial_statement': ('fs_id', 'code', 'rpid', 'bsns_year'),
 'balance_sheet': ('bs_id', 'code', 'rpid', 'quarter', 'account', 'value'),
 'income_statement': ('is_id', 'code', 'rpid', 'quarter', 'account', 'value')}

In [34]:
# Scenario 1 query
with psycopg.connect(
        dbname=db_settings['dbname'],
        host=db_settings['host'],
        user=db_settings['user'], 
        password=db_settings['password'],
        port=db_settings['port']
    ) as conn:
    with conn.cursor() as cur:
        cur.execute(
            """
            SELECT (1.0 * CAST(A.revenue AS REAL)) / (1.0 * CAST(B.cor AS REAL))
            FROM (
            (SELECT T.value AS revenue FROM vt_is_005930 AS T
            WHERE 
                T.bsns_year = '2019' AND 
                T.quarter = '4Q' AND 
                T.account = 'Revenue') AS A
            JOIN 
            (SELECT T.value AS cor FROM vt_is_005930 AS T
            WHERE 
                T.bsns_year = '2019' AND 
                T.quarter = '4Q' AND 
                T.account = 'CostOfSales') AS B ON 1=1
            )
            """
        )
        data = cur.fetchall()

data

[(-1.5648029930171061,)]

In [14]:
with psycopg.connect(
        dbname=db_settings['dbname'],
        host=db_settings['host'],
        user=db_settings['user'], 
        password=db_settings['password'],
        port=db_settings['port']
    ) as conn:
    with conn.cursor() as cur:
        cur.execute(
            """SELECT T.bsns_year, T.quarter, T.value FROM vt_is_005930 AS T
            WHERE T.account = 'Revenue'"""
        )
        data = cur.fetchall()

In [81]:
df_rev = pd.DataFrame(data, columns=['bsns_year', 'quarter', 'value'])
df_rev = df_rev.groupby(['bsns_year', 'quarter'])[['value']].first()
for bsns_year in range(2016, 2021):
    summ_to_3Q = df_rev.loc[(df_rev.index.get_level_values(0) == bsns_year) & (df_rev.index.get_level_values(1).isin(['1Q', '2Q', '3Q']))].sum()
    df_rev.loc[(bsns_year, '4Q')] = df_rev.loc[(bsns_year, '4Q')] - summ_to_3Q
df_rev['previous'] = df_rev.shift().values
df_rev = df_rev.dropna()
df_rev['previous'] = df_rev['previous'].values.astype(np.int64)
df_rev = df_rev.rename(columns={'value': 'current'})

In [73]:
df_rev.to_csv(src_path / 'data' / 'train_data.csv', encoding='utf-8')

In [80]:
df_rev = pd.read_csv(src_path / 'data' / 'train_data.csv', encoding='utf-8')
df_rev['date'] = df_rev['bsns_year'].astype(str) + '-' + df_rev['quarter']
df_rev2 = df_rev.loc[:, ['date', 'current']].set_index('date')
df_rev2.to_csv(src_path / 'data' / 'revenues.csv', encoding='utf-8')

In [82]:
df_rev_scaled = df_rev / 1e13 # 10,000,000,000,000  1e13
df_rev_scaled.to_csv(src_path / 'data' / 'train_data_scaled_1e13.csv', encoding='utf-8')

In [5]:
df_rev_scaled = pd.read_csv(src_path / 'data' / 'train_data_scaled_1e13.csv', encoding='utf-8')
df_rev_scaled = df_rev_scaled.set_index(keys=['bsns_year', 'quarter'])

In [6]:
X = df_rev_scaled.loc[:, 'previous'].values
y = df_rev_scaled.loc[:, 'current'].values

Linear Regression: https://towardsdatascience.com/how-to-model-time-series-data-with-linear-regression-cd94d1d901c0

In [4]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt

def pipeline(X, scale_factor=1, add_const=False):
    X = X / scale_factor
    if add_const:
        X = sm.add_constant(X, has_constant='add')
    return X

# model = LinearRegression()
# model.fit(pipeline(X), y)

In [7]:
model = sm.OLS(y, pipeline(X, add_const=True))
results = model.fit()

In [8]:
results.save(src_path / 'data' / 'linear_model.pickle')

In [132]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.283
Model:                            OLS   Adj. R-squared:                  0.247
Method:                 Least Squares   F-statistic:                     7.886
Date:                Fri, 17 Dec 2021   Prob (F-statistic):             0.0109
Time:                        00:54:10   Log-Likelihood:                -17.307
No. Observations:                  22   AIC:                             38.61
Df Residuals:                      20   BIC:                             40.80
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.5303      1.218      2.077      0.051      -0.011       5.071
x1             0.5845      0.208      2.808      0.011       0.150       1.019
==============================================================================
Omnibus:                        1.810   Durbin-Watson:                   2.029
Prob(Omnibus):                  0.405   Jarque-Bera (JB):                1.551
Skew:                           0.590   Prob(JB):                        0.461
Kurtosis:                       2.454   Cond. No.                         61.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [137]:
df_rev = pd.read_csv(src_path / 'data' / 'revenues.csv', encoding='utf-8').set_index('date')
df_rev['predicted'] = (results.predict(pipeline(df_rev_scaled['previous'].values, add_const=True)) * 1e13).astype(np.int64)
df_rev.to_csv(src_path / 'data' / 'revenues.csv', encoding='utf-8')

In [111]:
Q3_2021 = df_rev_scaled.loc[(2021, '3Q'), ['current']].values
results.predict(pipeline(Q3_2021, add_const=True))

array([6.85429466])

In [144]:
df_rev.loc['2021-3Q', ['current']]

current    73979187000000
Name: 2021-3Q, dtype: int64

In [112]:
model = sm.OLS(y, pipeline(X, add_const=False))
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.990
Model:                            OLS   Adj. R-squared (uncentered):              0.990
Method:                 Least Squares   F-statistic:                              2158.
Date:                Fri, 17 Dec 2021   Prob (F-statistic):                    1.17e-22
Time:                        00:44:21   Log-Likelihood:                         -19.456
No. Observations:                  22   AIC:                                      40.91
Df Residuals:                      21   BIC:                                      42.00
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.0147      0.022     46.452      0.000       0.969       1.060
==============================================================================
Omnibus:                        1.758   Durbin-Watson:                   2.532
Prob(Omnibus):                  0.415   Jarque-Bera (JB):                1.507
Skew:                           0.526   Prob(JB):                        0.471
Kurtosis:                       2.267   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

---